In [ ]:
from pyspark.sql import SparkSession

In [ ]:
from pyspark.sql import SparkSession
spark_session = SparkSession.builder.appName("MySparkApp").getOrCreate()

In [49]:
df = spark_session.read.csv("PBAL_Parc compteur_202409.csv", header=True, inferSchema=True, sep=";" )

In [50]:
df.head()

Row(PB='PB ALBOU', CONTRAT_RATTACHEMENT='AQUAWASH DT - 42257', COMMUNE='BOIGNY SUR BIONNE', CODE_CYCLE_SERVICE='0DYL            ', LIBELLE_CYCLE_SERVICE='BOIGNY SUR BIONNE SEMESTRIEL (Lot 1)', CODE_TOURNEE='0DYL            ', LIBELLE_TOURNEE_SERVICE='BOIGNY SUR BIONNE SEMESTRIEL (Lot 1)', TOURNEE_OPALE=1.0, SEQ_TOURNEE=9700, ID_COMPTEUR=8361561125, NUMERO_BADGE=1581412316, NUMERO_SERIE='H24VA169824', FABRICANT='Sappel - Diehl Metering - MID', MODELE='ALTAIR V4 composite T50', DIAMETRE=15, ANNEE_FABRICATION=2024, SOLUTION_COMPACTE='Oui', SOLUTION_DEPORTEE='Oui', PASSERELLE_SITE_ISOLE='Oui', DATE_RECEPTION='04/09/2024', DATE_DEPOSE=None, MATRICULE_EQUIPEMENT='11A5241011931007', ACCESSIBLE='OUI', CODE_EMPLACEMENT='AJ  ', LIBELLE_EMPLACEMENT='Jardin - Accessible', DETAILS_EMPLACEMENTS='JARDIN CHIENS', ID_PDS=6447593, DATE_MISE_EN_SERVICE_PDS='01/01/2009', ETAT_PDS='En service', ETAT_SOURCE_PDS='Ouvert', LOGEMENT_VACANT='NON      ', GEN_DIV='Normal       ', RACCORDE_RACORDABLE='Raccordable 

In [ ]:
df.printSchema()

root
 |-- PB: string (nullable = true)
 |-- CONTRAT_RATTACHEMENT: string (nullable = true)
 |-- COMMUNE: string (nullable = true)
 |-- CODE_CYCLE_SERVICE: string (nullable = true)
 |-- LIBELLE_CYCLE_SERVICE: string (nullable = true)
 |-- CODE_TOURNEE: string (nullable = true)
 |-- LIBELLE_TOURNEE_SERVICE: string (nullable = true)
 |-- TOURNEE_OPALE: double (nullable = true)
 |-- SEQ_TOURNEE: integer (nullable = true)
 |-- ID_COMPTEUR: long (nullable = true)
 |-- NUMERO_BADGE: integer (nullable = true)
 |-- NUMERO_SERIE: string (nullable = true)
 |-- FABRICANT: string (nullable = true)
 |-- MODELE: string (nullable = true)
 |-- DIAMETRE: integer (nullable = true)
 |-- ANNEE_FABRICATION: integer (nullable = true)
 |-- SOLUTION_COMPACTE: string (nullable = true)
 |-- SOLUTION_DEPORTEE: string (nullable = true)
 |-- PASSERELLE_SITE_ISOLE: string (nullable = true)
 |-- DATE_RECEPTION: string (nullable = true)
 |-- DATE_DEPOSE: string (nullable = true)
 |-- MATRICULE_EQUIPEMENT: string (null

In [51]:
#fonction qui parcours les colonne et transforme tous les colonne qui contiennent le mot "date" en objet date
from pyspark.sql.functions import col, to_date
def transform_date_columns(df):
    for column in df.columns:
        if "date" in column.lower():
            df = df.withColumn(column, to_date(col(column), "dd/MM/yyyy"))
    return df

In [52]:
df = transform_date_columns(df)
df.printSchema()

root
 |-- PB: string (nullable = true)
 |-- CONTRAT_RATTACHEMENT: string (nullable = true)
 |-- COMMUNE: string (nullable = true)
 |-- CODE_CYCLE_SERVICE: string (nullable = true)
 |-- LIBELLE_CYCLE_SERVICE: string (nullable = true)
 |-- CODE_TOURNEE: string (nullable = true)
 |-- LIBELLE_TOURNEE_SERVICE: string (nullable = true)
 |-- TOURNEE_OPALE: double (nullable = true)
 |-- SEQ_TOURNEE: integer (nullable = true)
 |-- ID_COMPTEUR: long (nullable = true)
 |-- NUMERO_BADGE: integer (nullable = true)
 |-- NUMERO_SERIE: string (nullable = true)
 |-- FABRICANT: string (nullable = true)
 |-- MODELE: string (nullable = true)
 |-- DIAMETRE: integer (nullable = true)
 |-- ANNEE_FABRICATION: integer (nullable = true)
 |-- SOLUTION_COMPACTE: string (nullable = true)
 |-- SOLUTION_DEPORTEE: string (nullable = true)
 |-- PASSERELLE_SITE_ISOLE: string (nullable = true)
 |-- DATE_RECEPTION: date (nullable = true)
 |-- DATE_DEPOSE: date (nullable = true)
 |-- MATRICULE_EQUIPEMENT: string (nullable

In [55]:
from pyspark.sql.functions import col, when, trim
def remove_spaces_and_replace_communes(df):
    communes_to_replace = ["ORLEANS", "SARAN"]
    df = df.select([
        when(trim(col(column)).isin(communes_to_replace), None)
        .otherwise(trim(col(column))).alias(column)
        if df.schema[column].dataType.typeName() == "string" else col(column)
        for column in df.columns
    ])

    return df

In [56]:
df1 = remove_spaces_and_replace_communes(df)
df1.printSchema()

root
 |-- PB: string (nullable = true)
 |-- CONTRAT_RATTACHEMENT: string (nullable = true)
 |-- COMMUNE: string (nullable = true)
 |-- CODE_CYCLE_SERVICE: string (nullable = true)
 |-- LIBELLE_CYCLE_SERVICE: string (nullable = true)
 |-- CODE_TOURNEE: string (nullable = true)
 |-- LIBELLE_TOURNEE_SERVICE: string (nullable = true)
 |-- TOURNEE_OPALE: double (nullable = true)
 |-- SEQ_TOURNEE: integer (nullable = true)
 |-- ID_COMPTEUR: long (nullable = true)
 |-- NUMERO_BADGE: integer (nullable = true)
 |-- NUMERO_SERIE: string (nullable = true)
 |-- FABRICANT: string (nullable = true)
 |-- MODELE: string (nullable = true)
 |-- DIAMETRE: integer (nullable = true)
 |-- ANNEE_FABRICATION: integer (nullable = true)
 |-- SOLUTION_COMPACTE: string (nullable = true)
 |-- SOLUTION_DEPORTEE: string (nullable = true)
 |-- PASSERELLE_SITE_ISOLE: string (nullable = true)
 |-- DATE_RECEPTION: date (nullable = true)
 |-- DATE_DEPOSE: date (nullable = true)
 |-- MATRICULE_EQUIPEMENT: string (nullable

In [57]:
#afficher les elemnent de la colonne commune "ORLEAN"
df1.filter(col("Commune") == "ORLEANS").show()

+---+--------------------+-------+------------------+---------------------+------------+-----------------------+-------------+-----------+-----------+------------+------------+---------+------+--------+-----------------+-----------------+-----------------+---------------------+--------------+-----------+--------------------+----------+----------------+-------------------+--------------------+------+------------------------+--------+---------------+---------------+-------+-------------------+-------------------+-----+------+-------------------+------------+-------------------+--------------+------------------+-------+------+-------------+------------+---------------+----------------+--------------+----------------+---------+----------------+-------------------+----------+-------------+--------------+-----------------+------------------+---+--------------------+-----------+---------------------+---------+---------+---------+---------+--------------------+---+------+-------+---------+----

In [58]:
df = remove_spaces_and_replace_communes(df)

In [59]:
#Transformer les variables diametre et INDEX_DERNIERE_RELEVE en integer et la variable CMJ en double
from pyspark.sql.functions import col, when
def transform_variables(df):
    df = df.withColumn("Diametre", col("Diametre").cast("integer"))
    df = df.withColumn("INDEX_DERNIERE_RELEVE", col("INDEX_DERNIERE_RELEVE").cast("integer"))
    df = df.withColumn("CMJ", col("CMJ").cast("double"))
    return df


In [60]:
df = transform_variables(df)

In [61]:
df.show()

+--------+--------------------+-----------------+------------------+---------------------+------------+-----------------------+-------------+-----------+-----------+------------+------------+--------------------+--------------------+--------+-----------------+-----------------+-----------------+---------------------+--------------+-----------+--------------------+----------+----------------+--------------------+--------------------+---------+------------------------+----------+---------------+---------------+-------------+--------------------+-------------------+------+----------+-------------------+------------+-------------------+--------------+------------------+---------+------+-------------+-----------------+---------------+----------------+--------------+----------------+---------+----------------+-------------------+----------+-------------+-----------------+-----------------+------------------+----+--------------------+------------------+---------------------+---------+--------

In [ ]:
#Afficher le nombre des element dont MODE_DE_RELEVE est "Télé-Relevé" et le nombre des element dont MODE_DE_RELEVE est "A pied" par COMMUNE
from pyspark.sql.functions import col, count, when
df.groupBy("Commune").pivot("MODE_DE_RELEVE").agg(count("*")).show()



+--------------------+----+------+------------+-----------+
|             Commune|null|A pied|Radio-Relevé|Télé-Relevé|
+--------------------+----+------+------------+-----------+
|              MARDIE|NULL|   314|           2|       1126|
|ST JEAN DE LA RUELLE|NULL|    28|          46|       NULL|
|                NULL|NULL|   621|          18|      23893|
|               ORMES|NULL|  1917|          73|         18|
|                 BOU|NULL|    87|        NULL|        457|
|  MARIGNY LES USAGES|NULL|   782|           1|         11|
|              OLIVET|NULL|   186|           1|       8058|
| SAINT JEAN LE BLANC|NULL|     2|        NULL|          1|
|    ST JEAN LE BLANC|NULL|    73|           9|       3278|
|               CHECY|   3|   647|           1|       3537|
|  ST PRYVE ST MESMIN|NULL|    47|           2|       2809|
|            CHANTEAU|NULL|   628|          25|          6|
|    ST JEAN DE BRAYE|NULL|    47|          44|          1|
|   BOIGNY SUR BIONNE|NULL|   588|      

In [28]:
#voir le nombre de ligne
df.count()

50319

In [62]:
#Supprimer les colonnes contenant uniquement des valeurs manquantes et renvoyer la listes des colonnes supprimer
def remove_all_null_columns_return_list(df):
    non_null_cols = [col for col in df.columns if df.filter(df[col].isNotNull()).count() > 0]
    columns_to_remove = [col for col in df.columns if col not in non_null_cols]
    return df.select(non_null_cols), columns_to_remove

In [63]:
#voir les colonnes supprimer
df, columns_to_remove = remove_all_null_columns_return_list(df)
print(columns_to_remove)

['FORAGE', 'ANC_A_FACTURER', 'BASSINS_VERSANTS', 'NOMBRE_UL', 'CODE_REM4', 'MO_RES', 'NAT_REJ', 'VOLUME_FORAGE_SITE', 'CODE_REF_AGENCE_EAU', 'L_REF_AGENCE_EAU', 'NB_LOG', 'NB_MIG', 'NB_PRO', 'NB_TOU', 'NB_UG1', 'NB_UG2', 'NB_UG3', 'NB_UG4', 'NB_UG5']


In [67]:
df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns]).show()

+---+--------------------+-------+------------------+---------------------+------------+-----------------------+-------------+-----------+-----------+------------+------------+---------+------+--------+-----------------+-----------------+-----------------+---------------------+--------------+-----------+--------------------+----------+----------------+-------------------+--------------------+------+------------------------+--------+---------------+---------------+-------+-------------------+-------------------+-----+------+-------------------+------------+-------------------+--------------+------------------+-------+-------------+------------+---------------+----------------+----------------+-------------------+----------+-------------+--------------+-----------------+------------------+-----+--------------------+-----------+---------------------+---------+---------+---------+--------------------+---+---------+---------+--------+-----------+---------------+-------------------+---------

In [71]:
#Compter les element dont l'année DATE_MISE_EN_SERVICE_PDS est 2023
from pyspark.sql.functions import col, year
df.filter(year(col("DATE_MISE_EN_SERVICE_PDS")) == 2023 ).count()

1169

In [72]:
#Afficher les element dont l'année DATE_MISE_EN_SERVICE_PDS est 2023 et le MODE_DE_RELEVE est "Télé-Relevé"
from pyspark.sql.functions import col, year
df.filter(
    (year(col("DATE_MISE_EN_SERVICE_PDS")) == 2023) &
    (col("MODE_DE_RELEVE") == "Télé-Relevé")
).show()

+--------+--------------------+-----------------+------------------+---------------------+------------+-----------------------+-------------+-----------+-----------+------------+------------+--------------------+--------------------+--------+-----------------+-----------------+-----------------+---------------------+--------------+-----------+--------------------+----------+----------------+--------------------+--------------------+----------+------------------------+----------+---------------+---------------+-------+--------------------+-------------------+------+----------+-------------------+------------+-------------------+--------------+------------------+----------+-------------+-----------------+---------------+----------------+----------------+-------------------+----------+-------------+-------------------+-----------------+------------------+----+--------------------+--------------------+---------------------+---------+---------+---------+--------------------+---+---------+--

In [73]:
#Afficher le nombre des element dont MODE_DE_RELEVE est "Télé-Relevé" et la Commune est "BOIGNY SUR BIONNE"
df.filter(col("MODE_DE_RELEVE")=="Télé-Relevé").filter(col("COMMUNE")=="BOIGNY SUR BIONNE").count()

502

In [74]:
df.filter((col("MODE_DE_RELEVE")=="Télé-Relevé") & (col("COMMUNE")=="BOIGNY SUR BIONNE")).count()

502

In [75]:
df.filter((col("MODE_DE_RELEVE")=="Télé-Relevé") & (col("COMMUNE")=="BOIGNY SUR BIONNE") & (col("Diametre") >= 40)).show()

+--------+--------------------+-----------------+------------------+---------------------+------------+-----------------------+-------------+-----------+-----------+------------+------------+--------------------+--------------------+--------+-----------------+-----------------+-----------------+---------------------+--------------+-----------+--------------------+----------+----------------+--------------------+--------------------+----------+------------------------+----------+---------------+---------------+-------+--------------------+-------------------+------+----------+-------------------+------------+-------------------+--------------+------------------+----------+-------------+-----------------+---------------+----------------+----------------+-------------------+----------+-------------+-----------------+-----------------+------------------+----+--------------------+-----------+---------------------+---------+---------+---------+--------------------+---+---------+-------------

In [76]:
#Nombre d'élément non null dans CONTRAT_RATTACHEMENT
df.filter(col("CONTRAT_RATTACHEMENT").isNotNull()).count()

50319

In [77]:
#Le nombre d'élément dans la colonne CONTRAT_RATTACHEMENT par COMMUNE
df.groupBy("COMMUNE").agg(count("CONTRAT_RATTACHEMENT")).show()

+--------------------+---------------------------+
|             COMMUNE|count(CONTRAT_RATTACHEMENT)|
+--------------------+---------------------------+
|              MARDIE|                       1442|
|                NULL|                      24532|
|                 BOU|                        544|
|  MARIGNY LES USAGES|                        794|
|              OLIVET|                       8245|
|               CHECY|                       4188|
|            CHANTEAU|                        659|
|   BOIGNY SUR BIONNE|                       1090|
|            COMBLEUX|                        304|
|ST JEAN DE LA RUELLE|                         74|
|               ORMES|                       2008|
| SAINT JEAN LE BLANC|                          3|
|    ST JEAN LE BLANC|                       3360|
|  ST PRYVE ST MESMIN|                       2858|
|    ST JEAN DE BRAYE|                         92|
|       ST CYR EN VAL|                        126|
+--------------------+---------

In [ ]:
#L'élément dans COMMUNE avec le plus d'élément de "Diamètre" suppérieur à 100
df.groupBy("COMMUNE").agg(count(col("Diamètre") >100))

In [81]:
df.groupBy("COMMUNE").agg(count(when(col("Diametre") > 100, 1)).alias("Count_Diamètre_GT_100")).orderBy(col("Count_Diamètre_GT_100").desc()).show()

+--------------------+---------------------+
|             COMMUNE|Count_Diamètre_GT_100|
+--------------------+---------------------+
|                NULL|                   35|
|       ST CYR EN VAL|                    5|
|               ORMES|                    3|
|              MARDIE|                    2|
|              OLIVET|                    2|
|  ST PRYVE ST MESMIN|                    2|
|               CHECY|                    1|
|   BOIGNY SUR BIONNE|                    1|
|            COMBLEUX|                    1|
|                 BOU|                    0|
|  MARIGNY LES USAGES|                    0|
|            CHANTEAU|                    0|
|ST JEAN DE LA RUELLE|                    0|
| SAINT JEAN LE BLANC|                    0|
|    ST JEAN LE BLANC|                    0|
|    ST JEAN DE BRAYE|                    0|
+--------------------+---------------------+

